In [111]:
#so far applying stopwords give more radicle results where as without stopwords gives more subtle results
#however, stopword gave 2 incorrect results while without stopwords gave 1 incorrect result

In [62]:
import warnings
warnings.filterwarnings("ignore")

In [99]:
from sentence_transformers import SentenceTransformer, util
import json
import numpy as np

import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
with open("./data/sample-data.json", "r") as f:
    data = json.load(f)

In [101]:
data.keys()

dict_keys(['coding', 'entertainment', 'finance', 'notes', 'quotes'])

In [102]:
new_data=data.copy()
stop_words = set(stopwords.words('english'))

for key, texts in new_data.items():
    filtered_texts = []
    for text in texts:
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        filtered_texts.append(' '.join(filtered_words))
    new_data[key] = filtered_texts

print(new_data)

{'coding': ['current sorting algorithm (Merge Sort) works well larger datasets, Quick Sort might perform better smaller datasets due lower overhead. Test confirm.', 'Binary Search Tree operations (insert, search, delete) O(log n) average, degrade O(n) tree becomes unbalanced. Consider using AVL Red-Black trees.', "'two-sum' problem, handle cases array contains duplicate numbers (e.g., [2, 2, 3] target 4).", "Use BFS shortest path unweighted graph Dijkstra's Algorithm weighted graphs. Avoid DFS may guarantee shortest path.", 'Extract repetitive API calls reusable function. reduce redundancy improve maintainability.', 'Switched axios fetch front-end reduces bundle size. Keep eye edge cases fetch lacks features (e.g., timeout).', 'Upgraded React v16 v18. Ensure third-party components support new React version.', 'Bug: NullPointerException calculateTotal(). Cause: order.items null. Fix: Add null check looping items.', 'Dropdown menu close blur Safari. Works Chrome/Firefox. Solution: Add ex

In [103]:
import random

for k,v in new_data.items():
    print(k, len(v))


coding 12
entertainment 10
finance 10
notes 8
quotes 10


In [104]:
train_data = {}
test_data = {}

for k, v in new_data.items():
    test_data[k] = random.sample(v, 2)
    train_data[k] = [item for item in v if item not in test_data[k]]

print("-----------train------------")
for k,v in train_data.items():
    print(k, len(v))
print("-----------test------------")
for k,v in test_data.items():
    print(k, len(v))

-----------train------------
coding 10
entertainment 8
finance 8
notes 6
quotes 8
-----------test------------
coding 2
entertainment 2
finance 2
notes 2
quotes 2


In [105]:
model= SentenceTransformer("all-mpnet-base-v2")


In [ ]:
# import time
# t = time.time()
# x=[]
# for i in range(100):
#     embeddings = model.encode(["sentences"])
#     x.append(embeddings[0])
# print(round((time.time()-t)*1000))

2515


In [107]:
class TextEmbeddingCentroidAnalyzer:
    def __init__(self, model_name='all-mpnet-base-v2'):
        self.model = SentenceTransformer(model_name)
    
    def generate_embeddings(self, texts):
        return self.model.encode(texts, convert_to_numpy=True)
    
    def calculate_class_centroids(self, texts, labels):
        embeddings = self.generate_embeddings(texts)
        centroids = {}
        for unique_class in np.unique(labels):
            class_mask = np.array(labels) == unique_class
            class_embeddings = embeddings[class_mask]
            centroid = np.mean(class_embeddings, axis=0)
            centroid = centroid / np.linalg.norm(centroid)
            centroids[unique_class] = centroid
        
        return centroids
    
    def find_nearest_centroid(self, text, centroids):
        text_embedding = self.generate_embeddings([text])[0]
        similarities = {}
        for class_name, centroid in centroids.items():
            similarity = np.dot(text_embedding, centroid)
            similarities[class_name] = similarity
        return similarities,
        # return max(similarities, key=similarities.get)


In [108]:
texts = []
labels = []
for k, v in train_data.items():
    for item in v:
        texts.append(item)
        labels.append(k)

In [109]:
analyzer = TextEmbeddingCentroidAnalyzer()

centroids = analyzer.calculate_class_centroids(texts, labels)

for class_name, centroid in centroids.items():
    print(f"Centroid for {class_name}: {centroid[:5]}...")

Centroid for coding: [ 0.01044333  0.05445974 -0.0276557  -0.00445685  0.01611437]...
Centroid for entertainment: [ 0.04053596  0.0366516  -0.01215939 -0.00458674  0.02963256]...
Centroid for finance: [-0.00610016  0.05527804 -0.0378691  -0.02249573  0.0577959 ]...
Centroid for notes: [ 0.01989413  0.07713379 -0.05561051 -0.00539542 -0.0045892 ]...
Centroid for quotes: [ 0.09016934  0.11804867 -0.00800988 -0.01948396  0.06845009]...


In [ ]:
# test_text = "movie"
# predicted_class = analyzer.find_nearest_centroid(test_text, centroids)
# v = np.array([k for k in predicted_class[0].values()])
# softmax_v = np.exp(v) / np.sum(np.exp(v))

# for i,cat in enumerate(predicted_class[0]):
#     print(cat, round(softmax_v[i].item()*100,2))

coding 18.73
entertainment 22.5
finance 16.94
notes 20.27
quotes 21.55


In [ ]:
for k, v in test_data.items():
    for text in v:
        predicted_class = analyzer.find_nearest_centroid(text, centroids)
        v = np.array([k for k in predicted_class[0].values()])
        # softmax_v = np.exp(v) / np.sum(np.exp(v))
        # print(k,end="->")
        # for i,cat in enumerate(predicted_class[0]):
        #     print(cat, round(softmax_v[i].item()*100,2),end=" ")
        # print("\n___\n")
        print(k,end="->")
        for i,cat in enumerate(predicted_class[0]):
            print(cat, round(v[i].item()*100,2),end=" ")
        print()
       

coding | Binary Search Tree operations (insert, s |->coding 32.54 entertainment -5.45 finance 4.01 notes -2.47 quotes -4.74 
coding | Switched from axios to fetch in the fron |->coding 26.15 entertainment 2.38 finance 4.63 notes 2.75 quotes 1.42 
entertainment | In Elden Ring, prioritize upgrading your |->coding 4.64 entertainment 15.12 finance 21.97 notes 8.34 quotes 10.12 
entertainment | Keep a list of movie s/web series to wat |->coding 10.29 entertainment 19.92 finance 26.99 notes 29.6 quotes 20.91 
finance | Review your credit report annually to ch |->coding 10.82 entertainment 4.53 finance 41.28 notes 13.05 quotes 6.36 
finance | Diversify your investment portfolio to r |->coding 15.43 entertainment 11.36 finance 73.29 notes 24.61 quotes 19.04 
notes | Travel Itinerary - Europe Trip
Destinati |->coding 5.01 entertainment 9.62 finance 2.99 notes 22.68 quotes 4.79 
notes | Workout Plan
Monday:
Chest and Triceps
T |->coding 7.35 entertainment 9.4 finance 16.81 notes 33.2 quotes 2.8

In [110]:
for k, v in test_data.items():
    for text in v:
        predicted_class = analyzer.find_nearest_centroid(text, centroids)
        v = np.array([k for k in predicted_class[0].values()])
        # softmax_v = np.exp(v) / np.sum(np.exp(v))
        # print(k,end="->")
        # for i,cat in enumerate(predicted_class[0]):
        #     print(cat, round(softmax_v[i].item()*100,2),end=" ")
        # print("\n___\n")
        print(k,end="->")
        for i,cat in enumerate(predicted_class[0]):
            print(cat, round(v[i].item()*100,2),end=" ")
        print()
       

coding->coding 20.27 entertainment -1.7 finance 11.87 notes 9.72 quotes 4.28 
coding->coding 21.41 entertainment 5.27 finance 19.85 notes 9.73 quotes 8.85 
entertainment->coding 4.1 entertainment 33.09 finance 13.58 notes 10.88 quotes 13.57 
entertainment->coding 7.28 entertainment 18.37 finance 23.07 notes 11.64 quotes 11.61 
finance->coding 12.41 entertainment 13.52 finance 40.65 notes 15.2 quotes 6.54 
finance->coding 11.42 entertainment 23.34 finance 59.19 notes 17.67 quotes 16.72 
notes->coding 5.55 entertainment 26.89 finance 18.44 notes 34.35 quotes 10.71 
notes->coding -5.17 entertainment -7.4 finance 3.1 notes 6.05 quotes -7.56 
quotes->coding -3.08 entertainment 26.48 finance 1.21 notes 7.67 quotes 49.86 
quotes->coding 9.69 entertainment 18.57 finance 19.96 notes 17.23 quotes 66.37 


coding->coding 20.27 entertainment -1.7 finance 11.87 notes 9.72 quotes 4.28 
coding->coding 21.41 entertainment 5.27 finance 19.85 notes 9.73 quotes 8.85 
entertainment->coding 4.1 entertainment 33.09 finance 13.58 notes 10.88 quotes 13.57 
entertainment->coding 7.28 entertainment 18.37 finance 23.07 notes 11.64 quotes 11.61 
finance->coding 12.41 entertainment 13.52 finance 40.65 notes 15.2 quotes 6.54 
finance->coding 11.42 entertainment 23.34 finance 59.19 notes 17.67 quotes 16.72 
notes->coding 5.55 entertainment 26.89 finance 18.44 notes 34.35 quotes 10.71 
notes->coding -5.17 entertainment -7.4 finance 3.1 notes 6.05 quotes -7.56 
quotes->coding -3.08 entertainment 26.48 finance 1.21 notes 7.67 quotes 49.86 
quotes->coding 9.69 entertainment 18.57 finance 19.96 notes 17.23 quotes 66.37

coding ->coding 32.54 entertainment -5.45 finance 4.01 notes -2.47 quotes -4.74 
coding ->coding 26.15 entertainment 2.38 finance 4.63 notes 2.75 quotes 1.42 
entertainment ->coding 4.64 entertainment 15.12 finance 21.97 notes 8.34 quotes 10.12 
entertainment ->coding 10.29 entertainment 19.92 finance 26.99 notes 29.6 quotes 20.91 
finance ->coding 10.82 entertainment 4.53 finance 41.28 notes 13.05 quotes 6.36 
finance ->coding 15.43 entertainment 11.36 finance 73.29 notes 24.61 quotes 19.04 
notes ->coding 5.01 entertainment 9.62 finance 2.99 notes 22.68 quotes 4.79 
notes ->coding 7.35 entertainment 9.4 finance 16.81 notes 33.2 quotes 2.8 
quotes ->coding -9.07 entertainment 0.2 finance 1.86 notes 5.83 quotes 17.41 
quotes ->coding 3.78 entertainment 25.0 finance 21.34 notes 20.67 quotes 72.63 